# SBM Estimation of Type Allocation

### Sink Restrictions, Data exports

James Yu

In [1]:
using Pkg
Pkg.activate(".")
# for pkg in ["HTTP", "JSON", "Distributions", "PrettyTables", "JLD", "IterTools"]
#     Pkg.add(pkg)
# end

#Pkg.instantiate()

  Activating project at `C:\Users\jbrig\Documents\research\mapinator_2024`


In [2]:
using Base.Threads, Random, SparseArrays, Distributions, JLD, IterTools

In [3]:
include("type_allocation_base.jl")

Main.SBM

In [4]:
to_from_by_year_api = SBM.fetch_data("to_from_by_year_mapinator_api.json")

Dict{String, Any} with 24 entries:
  "2009" => Dict{String, Any}("1886"=>Dict{String, Any}("disappeared"=>nothing,…
  "2018" => Dict{String, Any}("37196"=>Dict{String, Any}("disappeared"=>nothing…
  "2014" => Dict{String, Any}("55893"=>Dict{String, Any}("disappeared"=>0, "to_…
  "2005" => Dict{String, Any}("11382"=>Dict{String, Any}("disappeared"=>0, "to_…
  "2008" => Dict{String, Any}("3720"=>Dict{String, Any}("disappeared"=>0, "to_s…
  "2010" => Dict{String, Any}("5422"=>Dict{String, Any}("disappeared"=>nothing,…
  "2016" => Dict{String, Any}("29795"=>Dict{String, Any}("disappeared"=>nothing…
  "2019" => Dict{String, Any}("43054"=>Dict{String, Any}("disappeared"=>nothing…
  "2004" => Dict{String, Any}("24943"=>Dict{String, Any}("disappeared"=>nothing…
  "2026" => Dict{String, Any}("61387"=>Dict{String, Any}("disappeared"=>0, "to_…
  "2023" => Dict{String, Any}("56205"=>Dict{String, Any}("disappeared"=>0, "to_…
  "2007" => Dict{String, Any}("24350"=>Dict{String, Any}("disappeared"=>no

In [5]:
by_year_893 = SBM.fetch_data("to_from_by_year_893_mapinator_api.json")

Dict{String, Any} with 21 entries:
  "2014" => Dict{String, Any}("21269"=>Dict{String, Any}("disappeared"=>0, "to_…
  "2018" => Dict{String, Any}("30648"=>Dict{String, Any}("disappeared"=>0, "to_…
  "2005" => Dict{String, Any}("52068"=>Dict{String, Any}("disappeared"=>0, "to_…
  "2008" => Dict{String, Any}("13775"=>Dict{String, Any}("disappeared"=>0, "to_…
  "2010" => Dict{String, Any}("1495"=>Dict{String, Any}("disappeared"=>0, "to_s…
  "2016" => Dict{String, Any}("27274"=>Dict{String, Any}("disappeared"=>0, "to_…
  "2019" => Dict{String, Any}("42525"=>Dict{String, Any}("disappeared"=>0, "to_…
  "2007" => Dict{String, Any}("32"=>Dict{String, Any}("disappeared"=>0, "to_sho…
  "2023" => Dict{String, Any}("41022"=>Dict{String, Any}("disappeared"=>1, "to_…
  "2017" => Dict{String, Any}("40521"=>Dict{String, Any}("disappeared"=>0, "to_…
  "2022" => Dict{String, Any}("54879"=>Dict{String, Any}("disappeared"=>0, "to_…
  "2021" => Dict{String, Any}("48346"=>Dict{String, Any}("disappeared"=>0,

In [6]:
included_years = [parse(Int, year) for year in keys(to_from_by_year_api)]
# auto-detect year interval; change this to select the years of data to include in the estimation
YEAR_INTERVAL = minimum(included_years):maximum(included_years)

2003:2026

In [7]:
by_year_893_api = Dict()
for key in YEAR_INTERVAL
    if string(key) in keys(by_year_893)
        by_year_893_api[string(key)] = by_year_893[string(key)]
    end
end
by_year_893_api

Dict{Any, Any} with 20 entries:
  "2014" => Dict{String, Any}("21269"=>Dict{String, Any}("disappeared"=>0, "to_…
  "2005" => Dict{String, Any}("52068"=>Dict{String, Any}("disappeared"=>0, "to_…
  "2018" => Dict{String, Any}("30648"=>Dict{String, Any}("disappeared"=>0, "to_…
  "2020" => Dict{String, Any}("51250"=>Dict{String, Any}("disappeared"=>0, "to_…
  "2010" => Dict{String, Any}("1495"=>Dict{String, Any}("disappeared"=>0, "to_s…
  "2016" => Dict{String, Any}("27274"=>Dict{String, Any}("disappeared"=>0, "to_…
  "2019" => Dict{String, Any}("42525"=>Dict{String, Any}("disappeared"=>0, "to_…
  "2007" => Dict{String, Any}("32"=>Dict{String, Any}("disappeared"=>0, "to_sho…
  "2023" => Dict{String, Any}("41022"=>Dict{String, Any}("disappeared"=>1, "to_…
  "2017" => Dict{String, Any}("40521"=>Dict{String, Any}("disappeared"=>0, "to_…
  "2022" => Dict{String, Any}("54879"=>Dict{String, Any}("disappeared"=>0, "to_…
  "2021" => Dict{String, Any}("48346"=>Dict{String, Any}("disappeared"=>0, "t

In [8]:
applicant_mapping, department_mapping, applicant_list, department_list, academic, academic_to, academic_builder, rough_sink_builder, applicant_department_mapping = SBM.get_builders(to_from_by_year_api, YEAR_INTERVAL);

In [9]:
# sink of teaching universities that do not graduate PhDs
# this must be constructed using academic placements, not pre-defined sink placements

teaching_universities = Set() 
for dept_name in academic_to
    if !(dept_name in academic)
        # the department hired an assistant professor but never graduated anyone
        push!(teaching_universities, dept_name)
    end
end

In [10]:
public_sector = ("Public Sector", Set())
private_sector = ("Private Sector", Set())
other_groups = ("Other Groups", Set())

postdocs = ("Postdocs", Set())
lecturers = ("Lecturers", Set())
other_academic = ("Other Academic", Set())

for outcome in rough_sink_builder
    if outcome["recruiter_type"] == 5 # government institution
        push!(public_sector[2], (string(outcome["to_name"], " ($(public_sector[1]))"), outcome))
    elseif outcome["recruiter_type"] in [6, 7] # private sector: for and not for profit
        push!(private_sector[2], (string(outcome["to_name"], " ($(private_sector[1]))"), outcome))
    elseif outcome["recruiter_type"] == 8 # international organizations, think tanks, assorted
        push!(other_groups[2], (string(outcome["to_name"], " ($(other_groups[1]))"), outcome))

    # some other examples
    # every example here must also have a corresponding sink Set() above, 
    #     and an entry in sinks_to_include below
   
    elseif outcome["postype"] == 6
        # postdocs that are not in the above (i.e. academic; not public, private, or other)
        # requires reconfiguration of the to_from_by_year loading code
        push!(postdocs[2], (string(outcome["to_name"], " ($(postdocs[1]))"), outcome))
    elseif outcome["postype"] in [5, 7]
        # lecturers that are not in the above
        # requires reconfiguration of the to_from_by_year loading code
        push!(lecturers[2], (string(outcome["to_name"], " ($(lecturers[1]))"), outcome))
    else
        # everything else including terminal academic positions
        # this sink can only be constructed as an "else" statement
        push!(other_academic[2], (string(outcome["to_name"], " ($(other_academic[1]))"), outcome))
    
    end
end

# TODO: if not including a sink results in a department/applicant having zero placements, should it still be included in the adjacency matrix?
# if we know a department exists but has no placements, is this part of the network structure? 
# removing the department means the department does not exist in the network

# sort to ensure consistent ordering
academic_list = sort(collect(academic))
teaching_list = sort(collect(teaching_universities))
# to be consistent with the original estimation, we only include these additional sinks:
sinks_to_include = (public_sector, private_sector, other_groups, lecturers)#, postdocs, lecturers, other_academic)

sink_mapping, sink_list, sink_builder, sinks, sink_labels = SBM.build_sinks(sinks_to_include, teaching_list)

NUMBER_OF_SINKS = length(sink_labels)
institutions = vcat(academic_list, sinks...)
println("$(length(academic_list)) academic departments, $(length(institutions)) total departments")

Including the following sinks:
 Public Sector
 Private Sector
 Other Groups
 Lecturers
 Teaching Universities
Total 5 sinks
909 academic departments, 2578 total departments


In [11]:
length(academic_builder) + length(sink_builder)

13404

In [12]:
out = SBM.get_adjacency_by_department(department_mapping, sink_mapping, academic, academic_builder, sink_builder);

Total 13404 Placements (found 13404 by sequence counting, 13404 by matrix sum)


In [13]:
sum(out)

13404

In [14]:
out

2578×909 Matrix{Int32}:
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  1  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  

In [15]:
length(applicant_list)

14742

In [16]:
length(department_list)

909

In [17]:
[length(s) for s in sinks]

5-element Vector{Int64}:
 166
 233
  39
 490
 741

In [18]:
registration_entries = SBM.fetch_data("registrants.json");
no_print_spacer = 1;

In [19]:
BLANKCOUNT_TOL = 5000
NUMBER_OF_ROUNDS = 1000

NUMBER_OF_TYPES = 4
numtotal = NUMBER_OF_TYPES + NUMBER_OF_SINKS

all_placement_rates = zeros(Int, NUMBER_OF_ROUNDS, numtotal, NUMBER_OF_TYPES)
all_unmatched = zeros(Int, NUMBER_OF_ROUNDS, NUMBER_OF_TYPES)
all_m_t_values = zeros(Int, NUMBER_OF_ROUNDS, NUMBER_OF_TYPES)
all_likelihoods = zeros(NUMBER_OF_ROUNDS)
all_allocations = zeros(NUMBER_OF_ROUNDS, length(department_list) + sum([length(s) for s in sinks]))

Random.seed!(0)

# this approach with random re-running is similar to that used in/associated with 
#     the SBM MCMC solver package associated with Peixoto (2014)
Threads.@threads for SEED in 1:NUMBER_OF_ROUNDS
    if SEED % 100 == 0 println("starting seed = $SEED") end
    est_obj4, est_alloc4 = SBM.doit(sparse(out), 0, length(department_list), nothing, [length(s) for s in sinks], NUMBER_OF_TYPES, numtotal, BLANKCOUNT_TOL, should_print = false)
    placement_rates4, counts4, sorted_allocation4, full_likelihood4 = SBM.get_allocation(est_alloc4, out, 0, NUMBER_OF_TYPES, numtotal, sort_tiers = 1)

    all_likelihoods[SEED] = full_likelihood4
    all_allocations[SEED, :] = sorted_allocation4[:]
    # map names to tiers for graduating departments in this allocation
    name_tier_mapping = Dict()
    for (i, sbm_type) in enumerate(sorted_allocation4)
        if i <= length(department_list) # graduating departments
            name_tier_mapping[department_list[i]] = sbm_type
        end
    end

    # get m_t values for this allocation using name-tier mapping
    for entry in registration_entries
        if entry["name"] in keys(name_tier_mapping)
            all_m_t_values[SEED, name_tier_mapping[entry["name"]]] += 1
        end
    end

    all_placement_rates[SEED, :, :] = placement_rates4[:, :]

    # get unmatched entries for this allocation
    for year in keys(by_year_893)
        if parse(Int, year) in YEAR_INTERVAL
            # ASSUMPTION: every applicant ID has only one placement
            for (applicant_id, outcome) in by_year_893[year]
                if outcome["from_institution_name"] in keys(name_tier_mapping)
                    from_tier = name_tier_mapping[outcome["from_institution_name"]]
                    all_unmatched[SEED, from_tier] += 1
                end
            end
        end
    end
    
    if SEED % 100 == 0 println("  finished seed $SEED with likelihood $full_likelihood4") end
end

best_likelihood = maximum(all_likelihoods)
best_seed = argmax(all_likelihoods)
best_allocation = all_allocations[best_seed, :]
best_placements = all_placement_rates[best_seed, :, :]
best_unmatched = all_unmatched[best_seed, :]
best_m_t_values = all_m_t_values[best_seed, :]
println("number of types $NUMBER_OF_TYPES, best likelihood $best_likelihood")
display(best_placements)
println()
println()

save("type_allocation_results_$(NUMBER_OF_TYPES).jld", "placement_rates", best_placements, "unmatched", best_unmatched, "m_t_values", best_m_t_values, "likelihood", best_likelihood, "round_number", best_seed)

starting seed = 900
  finished seed 900 with likelihood -59229.96397619472
starting seed = 800
  finished seed 800 with likelihood -59238.478050593854
starting seed = 700
  finished seed 700 with likelihood -59233.20489548359
starting seed = 600
  finished seed 600 with likelihood -58845.96163126919
starting seed = 500
  finished seed 500 with likelihood -59231.0196878052
starting seed = 400
  finished seed 400 with likelihood -59238.478050593854
starting seed = 300
  finished seed 300 with likelihood -59229.96397619472
starting seed = 200
  finished seed 200 with likelihood -59238.478050593854
starting seed = 100
  finished seed 100 with likelihood -59229.859333156535
starting seed = 1000
  finished seed 1000 with likelihood -59237.00386943825
number of types 4, best likelihood -58835.856801263915


9×4 Matrix{Int64}:
  880   141   56    8
  938   547  158   31
 1147  1320  703   72
   95   220  247  211
  600   565  331   81
  727   428  243   76
  215   190  120   42
  149   245  349  287
  425   634  655  268

In [20]:
SBM.nice_table(best_placements, NUMBER_OF_TYPES, NUMBER_OF_SINKS, sink_labels)

┌───────────────────────┬────────┬────────┬────────┬────────┬────────────┐
│                       │ Tier 1 │ Tier 2 │ Tier 3 │ Tier 4 │ Row Totals │
├───────────────────────┼────────┼────────┼────────┼────────┼────────────┤
│                Tier 1 │    880 │    141 │     56 │      8 │       1085 │
│                Tier 2 │    938 │    547 │    158 │     31 │       1674 │
│                Tier 3 │   1147 │   1320 │    703 │     72 │       3242 │
│                Tier 4 │     95 │    220 │    247 │    211 │        773 │
│         Public Sector │    600 │    565 │    331 │     81 │       1577 │
│        Private Sector │    727 │    428 │    243 │     76 │       1474 │
│          Other Groups │    215 │    190 │    120 │     42 │        567 │
│             Lecturers │    149 │    245 │    349 │    287 │       1030 │
│ Teaching Universities │    425 │    634 │    655 │    268 │       1982 │
│         Column Totals │   5176 │   4290 │   2862 │   1076 │      13404 │
└───────────────────────┴

In [21]:
best_name_tier_mapping = Dict()
for (i, sbm_type) in enumerate(best_allocation)
    if sbm_type <= NUMBER_OF_TYPES
        best_name_tier_mapping[department_list[i]] = sbm_type
    else
        best_name_tier_mapping[sink_list[i - length(department_list)]] = sbm_type
    end
end

save("type_allocation_name_tier_mapping_$(NUMBER_OF_TYPES).jld", "name_tier_mapping", best_name_tier_mapping) 

In [22]:
for sorted_type in 1:numtotal
    dept_counter = 0
    inst_hold = []
    println("TYPE $sorted_type:")
    for (i, sbm_type) in enumerate(best_allocation)
        if sbm_type == sorted_type
            if sorted_type <= NUMBER_OF_TYPES
                push!(inst_hold, department_list[i])
            else
                push!(inst_hold, sink_list[i - length(department_list)])
            end
            dept_counter += 1
        end
    end
    for inst in sort(inst_hold)
        println("  ", inst)
    end
    println("Total Departments: $dept_counter")
    println()
    println()
end

TYPE 1:
  Boston University
  Columbia University
  Cornell University
  Duke University
  Harvard University
  London School of Economics and Political Science
  Massachusetts Institute of Technology
  New York University
  Northwestern University
  Pennsylvania State University
  Princeton University
  Stanford University
  University of California Los Angeles (UCLA)
  University of California, Berkeley
  University of Chicago
  University of Maryland
  University of Michigan
  University of Pennsylvania
  University of Toronto
  University of Wisconsin, Madison
  Yale University
Total Departments: 21


TYPE 2:
  Arizona State University
  Bocconi University
  Boston College
  Brown University
  Carnegie Mellon University
  Erasmus University Rotterdam
  European University Institute
  George Washington University
  Georgetown University
  Hong Kong University of Science and Technology
  Indiana University Bloomington
  Johns Hopkins University
  London Business School
  Maastricht U